In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

from ICARUS.core.types import FloatArray
from ICARUS.vehicle import SymmetryAxes
from ICARUS.vehicle import Airplane
from ICARUS.vehicle import WingSegment
from ICARUS.airfoils import NACA4
from ICARUS.computation.solvers.AVL import avl_polars
from ICARUS.environment import EARTH_ISA
from ICARUS.flight_dynamics import State
from ICARUS.vehicle import Elevator, Flap

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)

In [ ]:
origin = np.array([0.0, 0.0, 0.0], dtype=float)

wing_position = np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)
wing_orientation = np.array(
    [-1.0, 1.0, 0.0],
    dtype=float,
)

main_wing_flap = Flap(
    span_positions=(0.0, 0.35),
    hinge_chord_percentages=(0.75, 0.75),
    chord_extension=1.0,
)


main_wing = WingSegment(
    name="wing",
    root_airfoil=NACA4.from_digits("4415"),
    origin=origin + wing_position,
    orientation=wing_orientation,
    symmetries=SymmetryAxes.Y,
    span=2 * 1.130,
    sweep_offset=0,
    root_chord=0.159,
    tip_chord=0.072,
    twist_root=0,
    twist_tip=0,
    N=15,
    M=5,
    mass=0.670,
    controls=[main_wing_flap],
)

In [ ]:
elevator_pos: FloatArray = np.array(
    [0.54, 0.0, 0.0],
    dtype=float,
)
elevator_orientantion: FloatArray = np.array(
    [0.0, 0.0, 0.0],
    dtype=float,
)

elevator = Elevator(span_positions=(0.0, 1.0), hinge_chord_percentages=(0.5, 0.5))

hor_stabilizer = WingSegment(
    name="elevator",
    root_airfoil=NACA4.from_digits("0008"),
    origin=origin + elevator_pos,
    orientation=elevator_orientantion,
    symmetries=SymmetryAxes.Y,
    span=2 * 0.169,
    sweep_offset=0,
    root_dihedral_angle=0,
    root_chord=0.130,
    tip_chord=0.03,
    N=15,
    M=5,
    mass=0.06,
    controls=[elevator],
)

In [ ]:
rudder_position: FloatArray = np.array(
    [0.47, 0.0, 0.01],
    dtype=float,
)
rudder_orientation: FloatArray = np.array(
    [0.0, 0.0, 90.0],
    dtype=float,
)

vertical_stabilizer = WingSegment(
    name="rudder",
    root_airfoil=NACA4.from_digits("0008"),
    origin=origin + rudder_position,
    orientation=rudder_orientation,
    symmetries=SymmetryAxes.NONE,
    span=0.160,
    sweep_offset=0.1,
    root_dihedral_angle=0,
    root_chord=0.2,
    tip_chord=0.1,
    N=15,
    M=5,
    mass=0.04,
)

plane = Airplane("test", main_wing = main_wing, other_surfaces= [hor_stabilizer, vertical_stabilizer])

In [ ]:
from ICARUS.vehicle import Mass
point_masses = [
    Mass(mass= 0.500, position = np.array([-0.40, 0.0, 0.0], dtype=float), name = "engine"),  # Engine
    Mass(mass= 1.000, position = np.array([0.090, 0.0, 0.0], dtype=float), name = "structure"),  # Structure
    Mass(mass= 1.000, position = np.array([0.090, 0.0, 0.0], dtype=float), name = "battery"),  # Battery
    Mass(mass= 0.900, position = np.array([0.130, 0.0, 0.0], dtype=float), name = "payload"),  # Payload
]
plane.add_point_masses(point_masses)

In [ ]:
plane.wing_segments[2][1].airfoils[0].plot()

In [ ]:
from ICARUS.flight_dynamics import TrimNotPossible
from ICARUS.flight_dynamics.trim import TrimOutsidePolars

UINF = 20
state = State(
    name="Unstick",
    airplane=plane,
    environment=EARTH_ISA,
    u_freestream=UINF,
)

for d in [0, 10, 20, 30, 40]:
    solver2D = "Xfoil"
    angles = np.linspace(-10, 10, 11)

    epsilons = {
        "u": 0.01,
        "w": 0.01,
        "q": 0.1,
        "theta": 0.01,
        "v": 0.01,
        "p": 0.1,
        "r": 0.1,
        "phi": 0.01,
    }
    # print(state.control_vars)
    state.set_control(
        {
            "delta_f": d,
            "delta_e": 0,
        }
    )

    state.add_all_pertrubations("Central", epsilons)
    try:
        avl_polars(plane=plane, state=state, solver2D="Xfoil", angles=angles)
    except (TrimNotPossible , TrimOutsidePolars):
        continue

In [ ]:
from ICARUS.visualization.airplane import plot_airplane_polars 
plot_airplane_polars([plane.name], prefixes=["AVL"])